In [ ]:
import pandas as pd
import numpy as np
import os
import re
import io
import nltk
from nltk.corpus import stopwords
import math
import gzip
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train = pd.read_csv("./Dataset/train.csv")
train = train[['qid','article_id','response']]

In [ ]:
test = pd.read_csv("./Dataset/test.csv")
test = test[['qid','article_id','response']]

In [ ]:
df = pd.concat([train, test])
articles = pd.read_csv('./Dataset/Published_online_articles.csv')
users = pd.read_csv('./Dataset/subscribers.csv')

In [ ]:
article_id=articles['article_id'].unique()

In [ ]:
np.savetxt('./Dataset/Challenge_Dataset/articles.txt',article_id,fmt="%s")
np.savetxt('./Dataset/Challenge_Dataset/related_article.txt',article_id,fmt="%s")

In [ ]:
topic_list = []
topics = articles['topic'].dropna()
for topic in topics:
    t = topic.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_list:
            topic_list.append(j)

In [ ]:
len(topic_list)

55

In [ ]:
with open("./Dataset/Challenge_Dataset/topics.txt",'w') as f:
    for i in topic_list:
        f.write(i+'\n')
f.close()

In [ ]:
topic_tag_list = []
topic_tags = articles['vstopic'].dropna()
for tag in topic_tags:
    t = tag.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_tag_list:
            topic_tag_list.append(j)

In [ ]:
with open("./Dataset/Challenge_Dataset/topic_tags.txt",'w') as f:
    for i in topic_tag_list:
        f.write(i+'\n')
f.close()

In [ ]:
vsproduct_list = []
products = articles['vsproduct'].dropna()
for product in products:
    t = product.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in vsproduct_list:
            vsproduct_list.append(j)

In [ ]:
product_list = []
products = articles['product'].dropna()
for product in products:
    t = product.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in product_list:
            product_list.append(j)

In [ ]:
with open("./Dataset/Challenge_Dataset/products.txt",'w') as f:
    for i in product_list:
        f.write(i+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/product_tags.txt",'w') as f:
    for i in vsproduct_list:
        f.write(i+'\n')
f.close()

In [ ]:
user_id = df['qid'].unique()

In [ ]:
np.savetxt('./Dataset/Challenge_Dataset/users.txt',user_id,fmt="%d")

In [ ]:
d = {}
for i in user_id:
    d[i]=list(df[df['qid']==i]['article_id'])

In [ ]:
recommended_together = []
for a_id in article_id:
    tmp = {}
    for k,v in d.items():
        if a_id in tmp:
            if a_id in v:
                tmp[a_id]=list(set(tmp[a_id]).intersection(set(v)))
        else:
            if a_id in v:
                tmp[a_id]=v
            else:
                tmp[a_id]=[]
    recommended_together.append(tmp)

In [ ]:
recommended_together

[{'tcm:526-10787': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-10787',
   'tcm:526-208086']},
 {'tcm:526-10838': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-10838',
   'tcm:526-208086']},
 {'tcm:526-10849': ['tcm:526-22867',
   'tcm:526-208086',
   'tcm:526-12174',
   'tcm:526-10849']},
 {'tcm:526-116442': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-116442',
   'tcm:526-208086']},
 {'tcm:526-12167': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-12167',
   'tcm:526-208086']},
 {'tcm:526-12173': []},
 {'tcm:526-12174': ['tcm:526-22867', 'tcm:526-12174', 'tcm:526-208086']},
 {'tcm:526-12180': ['tcm:526-12180',
   'tcm:526-149043',
   'tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-208086']},
 {'tcm:526-13782': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-13782',
   'tcm:526-208086']},
 {'tcm:526-149043': ['tcm:526-22867',
   'tcm:526-12174',
   'tcm:526-149043',
   'tcm:526-208086']},
 {'tcm:526-153236': ['tcm:526-116442',
   'tcm:526-153236',
   'tcm:526-22867

In [ ]:
d_response_article = {}
for i in user_id:
    d_response_article[i]=list(df[(df['qid']==i) & (df['response']==1)]['article_id'])

In [ ]:
also_response = {}
for a_id in article_id:
    l = []
    for k,v in d_response_article.items():
        if a_id in v:
            l+=v
    also_response[a_id]=list(set(l))

In [ ]:
response_together = []
for a_id in article_id:
    tmp = {}
    for k,v in d_response_article.items():
        if a_id in tmp:
            if a_id in v:
                tmp[a_id]=list(set(tmp[a_id]).intersection(set(v)))
        else:
            if a_id in v:
                tmp[a_id] = v
            else:
                tmp[a_id] = []
    response_together.append(tmp)

In [ ]:
word_count_dict = {}

In [ ]:
def word_dict_generate(text):
    word_dict = {}
    words = []
    for line in text:
        line = line.strip().lower()
        #line = re.sub(r"[^A-Za-z0-9()<>:;,.!?\'\"]"," ",line)
        line = re.sub(r","," ",line)
        line = re.sub(r";"," ",line)
        line = re.sub(r"\'"," ",line)
        line = re.sub(r":"," ",line)
        line = re.sub(r"!"," ",line)
        line = re.sub(r"\."," ",line)
        line = re.sub(r"\/"," ",line)
        line = re.sub(r"\?"," ", line)
        line = re.sub(r"[\\d]"," ",line)
        line = re.sub(r"\'\'"," ",line)
        line = re.sub(r"\("," ",line)
        line = re.sub(r"\)"," ",line)
        line = re.sub(r"\<"," ",line)
        line = re.sub(r"\>"," ",line)
        line = nltk.word_tokenize(line)
        for w in line:
            if w not in stopwords.words("english") and len(w)>3:
                if w not in word_dict:
                    word_dict[w]=1
                else:
                    word_dict[w]+=1
                words.append(w)
        #words+=line
    return word_dict,words

In [ ]:
word_count_dict,words = word_dict_generate(articles['text'])

In [ ]:
len(word_count_dict),len(words)

(5359, 49263)

In [ ]:
def computeTF(wordDict, words):
    tfDict = {}
    wordsCount = len(words)
    for word, count in wordDict.items():
        tfDict[word] = count / float(wordsCount)
    return tfDict

In [ ]:
tf = computeTF(word_count_dict,words)

In [ ]:
def computeIDF(text):
    N = len(text)
    idfDict = dict.fromkeys(text.keys(), 0)
    for word, val in text.items():
        if val > 0:
            idfDict[word] += 1
    for word, val in idfDict.items():
        if val!=0:
            idfDict[word] = math.log(N / float(val))
    return idfDict

In [ ]:
idf = computeIDF(word_count_dict)

In [ ]:
def computeTFIDF(tfDict, idfs):
    tfidf = {}
    for word, val in tfDict.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [ ]:
tfidf = computeTFIDF(tf,idf)

In [ ]:
word_list = []
for w,s in tfidf.items():
    if s>0.0005:
        word_list.append(w)

In [ ]:
len(word_list)

2246

In [ ]:
with open("./Dataset/Challenge_Dataset/vocab.txt",'w') as f:
    for w in word_list:
        f.write(w+'\n')
f.close()

In [ ]:
def index_set(s):
    i = 0
    s_map = {}
    for key in s:
        s_map[key] = str(i)
        i+=1
    return s_map

In [ ]:
user_map = index_set(user_id)
article_map = index_set(article_id)
word_map = index_set(word_list)
topic_map = index_set(topic_list)
tag_map = index_set(topic_tag_list)
product_map = index_set(product_list)
topic_tag_map = index_set(topic_tag_list)
product_tag_map = index_set(vsproduct_list)

In [ ]:
with open("./Dataset/Challenge_Dataset/word_map.txt",'w') as f:
    for k,v in word_map.items():
        f.write(k+' '+v+'\n')
    f.close()
with open("./Dataset/Challenge_Dataset/topic_map.txt",'w') as f:
    for k,v in topic_map.items():
        f.write(k+' '+v+'\n')
    f.close()

with open("./Dataset/Challenge_Dataset/product_map.txt",'w') as f:
    for k,v in product_map.items():
        f.write(k+' '+v+'\n')
    f.close()

with open("./Dataset/Challenge_Dataset/topic_tag_map.txt",'w') as f:
    for k,v in topic_tag_map.items():
        f.write(k+' '+v+'\n')
    f.close()

with open("./Dataset/Challenge_Dataset/product_tag_map.txt",'w') as f:
    for k,v in product_tag_map.items():
        f.write(k+' '+v+'\n')
    f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/response_together.txt",'w') as f:
    for line in response_together:
        t = ''
        for k,v in line.items():
            for k in v:
                t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/recommended_together.txt",'w') as f:
    for line in recommended_together:
        t = ''
        for k,v in line.items():
            for k in v:
                t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/also_response.txt",'w') as f:
    for k,v in also_response.items():
        t = ''
        for k in v:
            t=t+' '+article_map[k]
        f.write(article_map[k]+' '+t+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/has_topic.txt",'w') as f:
    for index,row in articles.iterrows():
        g = ''
        if type(row['topic']) == str:
            topics = row['topic'].strip('[').strip(']').strip().split(',')
            for t in topics:
                s = t.strip().replace("'",'')
                g=g+' '+topic_map[s]
        f.write(g+'\n')
f.close()

In [ ]:
#has_vstopic_tag
with open("./Dataset/Challenge_Dataset/has_topic_tag.txt",'w') as f:
    for index,row in articles.iterrows():
        k = ''
        if type(row['vstopic']) == str:
            tags = row['vstopic'].strip('[').strip(']').strip().split(',')
            for t in tags:
                s = t.strip().replace("'",'')
                k=k+' '+topic_tag_map[s]
        #f.write(article_map[row['tcm_id']]+' '+k+'\n')
        f.write(k+'\n')
f.close()

In [ ]:
#has_vsproduct_tag
with open("./Dataset/Challenge_Dataset/has_product_tag.txt",'w') as f:
    for index,row in articles.iterrows():
        k = ''
        if type(row['vsproduct']) == str:
            tags = row['vsproduct'].strip('[').strip(']').strip().split(',')
            for t in tags:
                s = t.strip().replace("'",'')
                k=k+' '+product_tag_map[s]
        #f.write(article_map[row['tcm_id']]+' '+k+'\n')
        f.write(k+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/has_product.txt",'w') as f:
    for index,row in articles.iterrows():
        g = ''
        #print(article_map[row['tcm_id']]+' '+
        if type(row['product']) == str:
            products = row['product'].strip('[').strip(']').strip().split(',')
            for t in products:
                s = t.strip().replace("'",'')
                g=g+' '+product_map[s]
        f.write(g+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/dataset.txt",'w') as f:
    for index,row in df.iterrows():
        if len(articles[articles['article_id']==row['article_id']]['text'].values) > 0:
            text = articles[articles['article_id']==row['article_id']]['text'].values[0].strip().lower().split(' ')
            k = ''
            for w in text:
                if w in word_map:
                    k=k+' '+word_map[w]
            f.write(user_map[row['qid']]+'\t'+article_map[row['article_id']]+'\t'+k+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/train.txt",'w') as f:
    for index,row in train.iterrows():
            if len(articles[articles['article_id']==row['article_id']]['text'].values) > 0:
                text = articles[articles['article_id']==row['article_id']]['text'].values[0].strip().lower().split(' ')
                k = ''
                for w in text:
                    if w in word_map:
                        k=k+' '+word_map[w]
                f.write(user_map[row['qid']]+'\t'+article_map[row['article_id']]+'\t'+k+'\n')
f.close()

In [ ]:
with open("./Dataset/Challenge_Dataset/test.txt",'w') as f:
    for index,row in test.iterrows():
        if len(articles[articles['article_id']==row['article_id']]['text'].values) > 0:
            text = articles[articles['article_id']==row['article_id']]['text'].values[0].strip().lower().split(' ')
            k = ''
            for w in text:
                if w in word_map:
                    k=k+' '+word_map[w]
            f.write(user_map[row['qid']]+'\t'+article_map[row['article_id']]+'\t'+k+'\n')
f.close()